# Studi Essentia: Introduzione all'elaborazione audio in Python

Questo notebook introduce i fondamenti della libreria Essentia per l'elaborazione audio in Python. Essentia è una libreria open-source sviluppata dal Music Technology Group dell'Universitat Pompeu Fabra, specializzata nell'analisi di segnali audio con un focus sulla musica.

## Panoramica
In questo notebook esploreremo:
- Come caricare e visualizzare file audio
- Come analizzare l'ampiezza di un segnale audio
- Come utilizzare algoritmi di inviluppo (envelope) per analizzare l'evoluzione dell'ampiezza nel tempo
- Come creare visualizzazioni interattive dei risultati

Iniziamo con l'installazione delle librerie necessarie.

In [33]:
# Installazione delle librerie necessarie
# Esegui questa cella solo se le librerie non sono già installate

'''
%pip install --upgrade pip
%pip install plotly
%pip install nbformat
%pip install ipywidgets
'''
!pip install nbformat


In [36]:
import nbformat
print(nbformat.__version__)

5.10.4


## 1. Importazione dei moduli necessari

Prima di iniziare, importiamo i moduli necessari per il nostro lavoro.

In [ ]:
# Importazione dei moduli principali
import essentia
import essentia.standard as es  # Modulo standard di Essentia
import numpy as np              # Per la manipolazione degli array
import plotly.graph_objects as go # Per visualizzazioni interattive
from plotly.subplots import make_subplots
import nbformat

## 2. Esplorazione della libreria Essentia

Essentia contiene un'ampia varietà di algoritmi per l'analisi audio. Possiamo esplorare cosa offre utilizzando la funzione `dir()`.

In [15]:
# Visualizziamo tutti gli algoritmi disponibili in essentia.standard
print(dir(es))

['AfterMaxToBeforeMaxEnergyRatio', 'AllPass', 'Audio2Midi', 'Audio2Pitch', 'AudioLoader', 'AudioOnsetsMarker', 'AudioWriter', 'AutoCorrelation', 'BFCC', 'BPF', 'BandPass', 'BandReject', 'BarkBands', 'BeatTrackerDegara', 'BeatTrackerMultiFeature', 'Beatogram', 'BeatsLoudness', 'BinaryOperator', 'BinaryOperatorStream', 'BpmHistogram', 'BpmHistogramDescriptors', 'BpmRubato', 'CartesianToPolar', 'CentralMoments', 'Centroid', 'ChordsDescriptors', 'ChordsDetection', 'ChordsDetectionBeats', 'ChromaCrossSimilarity', 'Chromagram', 'ClickDetector', 'Clipper', 'ConstantQ', 'CoverSongSimilarity', 'Crest', 'CrossCorrelation', 'CrossSimilarityMatrix', 'CubicSpline', 'DCRemoval', 'DCT', 'Danceability', 'Decrease', 'Derivative', 'DerivativeSFX', 'DiscontinuityDetector', 'Dissonance', 'DistributionShape', 'Duration', 'DynamicComplexity', 'ERBBands', 'EasyLoader', 'EffectiveDuration', 'Energy', 'EnergyBand', 'EnergyBandRatio', 'Entropy', 'Envelope', 'EqloudLoader', 'EqualLoudness', 'Extractor', 'FFT', '

### Esplorazione di specifici algoritmi

La libreria Essentia funziona accedendo ai singoli algoritmi presenti nel modulo `standard`. Per esempio, possiamo ottenere informazioni su un filtro passa-tutto (`AllPass`) o qualsiasi altro algoritmo.

In [16]:
# Visualizziamo i dettagli dell'algoritmo AllPass
es.AllPass

essentia.standard._create_essentia_class.<locals>.Algo

Per ottenere informazioni dettagliate su un algoritmo specifico, possiamo utilizzare la funzione `help()`:

In [17]:
# Visualizziamo la documentazione completa dell'algoritmo AllPass
help(es.AllPass)

Help on class Algo in module essentia.standard:

class Algo(Algorithm)
 |  Algo(**kwargs)
 |
 |  AllPass
 |
 |
 |  Inputs:
 |
 |    [vector_real] signal - the input signal
 |
 |
 |  Outputs:
 |
 |    [vector_real] signal - the filtered signal
 |
 |
 |  Parameters:
 |
 |    bandwidth:
 |      real ∈ (0,inf) (default = 500)
 |      the bandwidth of the filter [Hz] (used only for 2nd-order filters)
 |
 |    cutoffFrequency:
 |      real ∈ (0,inf) (default = 1500)
 |      the cutoff frequency for the filter [Hz]
 |
 |    order:
 |      integer ∈ {1,2} (default = 1)
 |      the order of the filter
 |
 |    sampleRate:
 |      real ∈ (0,inf) (default = 44100)
 |      the sampling rate of the audio signal [Hz]
 |
 |
 |  Description:
 |
 |    This algorithm implements a IIR all-pass filter of order 1 or 2. Because of
 |    its dependence on IIR, IIR's requirements are inherited.
 |
 |    References:
 |      [1] U. Zölzer, DAFX - Digital Audio Effects, p. 43,
 |      John Wiley & Sons, 2002
 |


## 3. Caricamento e visualizzazione di file audio

Ora carichiamo un file audio di esempio chiamato "0_01_9.wav" utilizzando l'algoritmo `AudioLoader`.

In [18]:
# Inizializziamo l'oggetto AudioLoader specificando il file da caricare
loader = es.AudioLoader(filename='dataset/originali/1.wav')

# Eseguiamo l'algoritmo e otteniamo i risultati
# Il metodo restituisce una tupla con diverse informazioni
audio, sr, n_channel, md5, bitrate, codec = loader()

### Analisi dei dati caricati

L'algoritmo `AudioLoader` restituisce diverse informazioni sul file audio caricato:
- `audio`: array contenente i campioni audio
- `sr`: frequenza di campionamento (sample rate) in Hz
- `n_channel`: numero di canali audio
- `md5`: hash MD5 del file
- `bitrate`: bit rate del file
- `codec`: codec utilizzato per la codifica del file

Verifichiamo la frequenza di campionamento:

In [19]:
# Visualizziamo la frequenza di campionamento
print(f"Frequenza di campionamento: {sr} Hz")
print(f"MD5: {loader()}")

Frequenza di campionamento: 48000.0 Hz
MD5: (array([[-0.00164795,  0.        ],
       [-0.00180054,  0.        ],
       [-0.0017395 ,  0.        ],
       ...,
       [ 0.00079346,  0.        ],
       [ 0.00091553,  0.        ],
       [ 0.00094604,  0.        ]], dtype=float32), 48000.0, 1, '', 768000, 'pcm_s16le')


### Gestione dei canali audio

I dati audio vengono caricati in un formato interlacciato. Se il file è stereo, avremo due canali; se è mono, il secondo canale sarà costituito da zeri. Dobbiamo estrapolare correttamente i dati per lavorare con essi.

In [20]:
# Visualizziamo i dati audio grezzi
print("Forma dell'array audio:", audio.shape)
audio

Forma dell'array audio: (27905, 2)


array([[-0.00164795,  0.        ],
       [-0.00180054,  0.        ],
       [-0.0017395 ,  0.        ],
       ...,
       [ 0.00079346,  0.        ],
       [ 0.00091553,  0.        ],
       [ 0.00094604,  0.        ]], dtype=float32)

In [21]:
# Trasponiamo la matrice per ottenere un array per ogni canale
audio_T = audio.T
print("Forma dopo la trasposizione:", audio_T.shape)

Forma dopo la trasposizione: (2, 27905)


In [22]:
# Estraiamo il primo canale (per file mono è l'unico con dati significativi)
audio = audio_T[0]
print("Lunghezza del segnale audio:", len(audio))

Lunghezza del segnale audio: 27905


### Ascolto del file audio

Possiamo utilizzare `IPython.display.Audio` per ascoltare il file direttamente nel notebook.

In [23]:
import IPython.display
IPython.display.Audio('./dataset/originali/1.wav', rate=sr)

### Visualizzazione della forma d'onda

Utilizziamo `plotly` per creare una visualizzazione interattiva della forma d'onda.

In [24]:
# Creazione di un grafico della forma d'onda
fig = go.Figure(data=go.Scatter(y=audio))
fig.update_layout(
    title="Forma d'onda del file audio",
    xaxis_title="Campioni",
    yaxis_title="Ampiezza"
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 4. Analisi dell'ampiezza

### Calcolo dell'inviluppo del segnale

L'inviluppo (envelope) è una rappresentazione dell'evoluzione dell'ampiezza di un segnale nel tempo. L'algoritmo `Envelope` di Essentia implementa un rilevatore di inviluppo basato su rettificazione e filtro passa-basso.

Parametri importanti dell'algoritmo `Envelope`:
- `attackTime`: tempo di attacco in millisecondi
- `releaseTime`: tempo di rilascio in millisecondi
- `sampleRate`: frequenza di campionamento del segnale

In [ ]:
# Calcoliamo l'inviluppo del segnale
env_op = es.Envelope(sampleRate=sr, releaseTime=10)  # Inizializziamo l'algoritmo
env = env_op(audio)                  # Applichiamo l'algoritmo al nostro segnale

print(f"Lunghezza dell'inviluppo: {len(env)} campioni")

### Visualizzazione dell'inviluppo

Visualizziamo l'inviluppo sovrapposto alla forma d'onda per capire come rappresenta l'evoluzione dell'ampiezza.

In [ ]:
# Creiamo un grafico con forma d'onda e inviluppo
fig = go.Figure()
fig.add_trace(go.Scatter(y=audio, name="Forma d'onda"))
fig.add_trace(go.Scatter(y=env, name="Inviluppo"))
fig.update_layout(
    title="Forma d'onda e inviluppo",
    xaxis_title="Campioni",
    yaxis_title="Ampiezza"
)
fig.show()

## 5. Widget interattivo per l'analisi dell'inviluppo

Creiamo un widget interattivo che ci permetta di regolare il tempo di rilascio dell'algoritmo di inviluppo e vedere come questo parametro influenza il risultato.

In [ ]:
import ipywidgets as widgets

def update_plot(rTime) -> None:
    """Funzione per aggiornare il grafico in base al tempo di rilascio selezionato"""
    # Calcolo dell'inviluppo con il nuovo tempo di rilascio
    env_op = es.Envelope(sampleRate=sr, releaseTime=rTime)
    env = env_op(audio)

    # Creazione del grafico aggiornato
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=audio, name="Forma d'onda"))
    fig.add_trace(go.Scatter(y=env, name="Inviluppo"))
    fig.update_layout(
        title=f"Inviluppo con tempo di rilascio: {rTime} ms",
        xaxis_title="Campioni",
        yaxis_title="Ampiezza"
    )
    
    fig.show()

# Creiamo uno slider per regolare il tempo di rilascio
release_slider = widgets.FloatSlider(
    min=1,
    max=120,
    value=10,
    description="Tempo di rilascio (ms)"
)

# Creiamo il widget interattivo
widgets.interactive(update_plot,rTime=release_slider)

## 6. Altri algoritmi di analisi dell'ampiezza

Oltre all'inviluppo, Essentia offre altri algoritmi per l'analisi dell'ampiezza, come `RMS` (Root Mean Square) per calcolare l'energia del segnale.

In [ ]:
# Calcoliamo il valore RMS del segnale
rms = es.RMS()
rms_value = rms(audio)
print(f"Valore RMS del segnale: {rms_value}")

In [26]:
# Calcoliamo il valore RMS in finestre di 100 campioni
window= 100
len(audio)/window


279.05

simili sono round(), ceil(), floor()

In [ ]:
a = [9,8,7,6,5,4,3,2,1]
a[0]

per leggere alcune porzioni di array invece

In [ ]:
a = [9,8,7,6,5,4,3,2,1]
print(a[2:]) #leggi da
print(a[:-2]) #leggi fino a
print(a[2:5]) #leggi da 2 a 5

In [27]:
for i in range(0, len(audio), window):
     # Calcoliamo il frame corrente
    frame = audio[i:i+window]
    print(len(frame))
   

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [28]:
rms_a = es.RMS()
# Creoamo una lista per memorizzare i valori RMS
rms = []
for i in range(0, len(audio), window):
    # Calcoliamo il valore RMS per il frame corrente
    frame = audio[i:i+window]
    rms_value = rms_a(frame)
    # Aggiungiamo il valore RMS alla lista
    rms.append(rms_value)
    print(f"RMS del frame {i//window}: {rms_value}")
print(len(rms))

RMS del frame 0: 0.0017811210127547383
RMS del frame 1: 0.0017493946943432093
RMS del frame 2: 0.001826537656597793
RMS del frame 3: 0.0016292185755446553
RMS del frame 4: 0.0015298441285267472
RMS del frame 5: 0.0013260545674711466
RMS del frame 6: 0.0013246422167867422
RMS del frame 7: 0.0010878563625738025
RMS del frame 8: 0.0007828576490283012
RMS del frame 9: 0.0004543841059785336
RMS del frame 10: 0.0003704587579704821
RMS del frame 11: 0.00020249970839358866
RMS del frame 12: 0.00013448526442516595
RMS del frame 13: 5.5437933042412624e-05
RMS del frame 14: 6.979124009376392e-05
RMS del frame 15: 0.00018424632435198873
RMS del frame 16: 0.0003516414435580373
RMS del frame 17: 0.00045801792293787
RMS del frame 18: 0.000742004020139575
RMS del frame 19: 0.0009997523156926036
RMS del frame 20: 0.0011068671010434628
RMS del frame 21: 0.001115378923714161
RMS del frame 22: 0.0010367580689489841
RMS del frame 23: 0.0010925207752734423
RMS del frame 24: 0.0009336035000160336
RMS del fra

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=audio, name="Forma d'onda"))
fig.add_trace(go.Scatter(y=rms, name="RMS"))
fig.show()

lo veidmao per pochi campioni, quindi distribuiamo il grafico dell'rms con numpy

In [ ]:
fig = go.Figure()
# Creiamo un asse temporale per il grafico 
time = np.linspace(0, len(audio), len(rms))
fig.add_trace(go.Scatter(y=audio, name="Forma d'onda"))
fig.add_trace(go.Scatter(x=time, y = rms, name="RMS"))
fig.show()

con frame generator abbiamo il contenuto della finestra, spiegare poi cosa è Hop size con grafici e 

### Utilizzo di `FrameGenerator` per l'analisi a finestre

Il codice sottostante utilizza l'algoritmo `FrameGenerator` di Essentia per suddividere il segnale audio in frame (finestre) di dimensione fissa. Questo approccio è utile per analizzare il segnale in segmenti temporali più piccoli, ad esempio per calcolare il valore RMS o altre caratteristiche su ogni frame.

#### Parametri principali di `FrameGenerator`:
- **`audio`**: il segnale audio da suddividere in frame.
- **`frameSize`**: la dimensione di ogni frame in campioni. Ad esempio, con una frequenza di campionamento di 44100 Hz, un frame di 1024 campioni corrisponde a circa 23 ms.
- **`hopSize`**: il numero di campioni di sovrapposizione tra un frame e il successivo. Un valore di 1 significa che i frame sono completamente sovrapposti, mentre un valore uguale a `frameSize` significa che i frame non si sovrappongono.
- **`startFromZero`** (opzionale): se impostato su `True`, il primo frame inizia dal campione 0. Il valore predefinito è `False`.

Il codice calcola il valore RMS per ogni frame generato e lo memorizza in una lista. Successivamente, viene creato un grafico che mostra la forma d'onda del segnale originale e i valori RMS calcolati per ogni frame.

In [ ]:
rms=[]
# Creiamo un oggetto FrameGenerator
for frame in es.FrameGenerator(audio, frameSize=1024, hopSize=1):
    rms.append(rms_a(frame))

fig = go.Figure()
time = np.linspace(0, len(audio), len(rms))
fig.add_trace(go.Scatter(y=audio, name="Forma d'onda"))
fig.add_trace(go.Scatter(x=time, y = rms, name="RMS"))
fig.show()


STFT ora

prendiamlo la funziona custom da utils/funzioni.py

In [ ]:
from utils.funzioni import create_sine

sinusoide = create_sine(300)
fig = go.Figure()
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"))
#window = 1024
#spect = es.Spectrum()  # Initialize the Spectrum

In [ ]:
from utils.funzioni import create_sine_es

sinusoide = create_sine_es(9100, length=1024)
fig = go.Figure()
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"))

window = 1024
spect = es.Spectrum(size = window)  # Initialize the Spectrum
sin_spec = spect(sinusoide)

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"), row=1, col=1)
fig.add_trace(go.Scatter(y=sin_spec, name = "Spettro"), row=2, col=1)
fig.show()


### Dimensione della finestra e risoluzione spettrale

Quando la dimensione della finestra (`window size`) è impostata a 64 campioni, questa scelta determina la risoluzione spettrale e temporale dell'analisi.

#### Relazione tra tempo e frequenza
Se sull'asse `x` rappresentiamo il tempo e sull'asse `y` la frequenza, la dimensione della finestra influenza direttamente la risoluzione in frequenza. La risoluzione spettrale è data dalla formula:

$\text{Risoluzione} = \frac{\text{Sample Rate}}{\text{Window Size}}$

Ad esempio, con una frequenza di campionamento ($ \text{Sample Rate}$) di 44100 Hz e una finestra di 64 campioni:

$\text{Risoluzione} = \frac{44100}{64} \approx 689.06 \, \text{Hz}$

Questo significa che ogni bin dello spettro rappresenta una banda di circa 689 Hz.

#### Relazione tra bin e frequenza
Se osserviamo un picco al bin \( n \), la frequenza corrispondente può essere calcolata moltiplicando il numero del bin per la risoluzione:

$\text{Frequenza} = n \cdot \text{Risoluzione}$

Ad esempio, se il picco si trova al bin 5, la frequenza corrispondente sarà:

$\text{Frequenza} = 5 \cdot 689.06 \, \text{Hz} \approx 3445.3 \, \text{Hz}$

#### Scelta della finestra e asse temporale
Con una finestra di 64 campioni, possiamo decidere che l'asse temporale rappresenterà 400 campioni. Questo significa che ogni finestra coprirà una porzione specifica del segnale, influenzando la granularità dell'analisi temporale.

In sintesi:
- La dimensione della finestra determina la risoluzione in frequenza.
- La relazione tra bin e frequenza è lineare e dipende dalla risoluzione.
- La scelta della finestra influenza sia l'asse temporale che quello spettrale.


In [ ]:
195*46

### Relazione tra il picco spettrale e la risoluzione spettrale

Nel codice precedente, abbiamo calcolato lo spettro di una sinusoide utilizzando una finestra di dimensione fissa. Analizziamo ora il valore del picco spettrale per verificare la relazione tra la frequenza del picco e la risoluzione spettrale. Questo ci aiuta a comprendere meglio come la dimensione della finestra influisce sulla precisione dell'analisi in frequenza.

#### Formula per la risoluzione spettrale
La risoluzione spettrale è definita come:

''' math
\text{Risoluzione} = \frac{\text{Sample Rate}}{\text{Window Size}}
'''

Dove:
- **Sample Rate** è la frequenza di campionamento del segnale (in Hz).
- **Window Size** è la dimensione della finestra (in campioni).

#### Calcolo della frequenza del picco
La frequenza corrispondente a un determinato bin \( n \) dello spettro può essere calcolata come:

''' math
\text{Frequenza} = n \cdot \text{Risoluzione}
'''

#### Esempio pratico
Supponiamo di avere:
- **Sample Rate**: 44100 Hz
- **Window Size**: 1024 campioni

La risoluzione spettrale sarà:

''' math
\text{Risoluzione} = \frac{44100}{1024} \approx 43.07 \, \text{Hz}
'''

Se il picco si trova al bin \( n = 210 \), la frequenza corrispondente sarà:

''' math
\text{Frequenza} = 210 \cdot 43.07 \approx 9034.7 \, \text{Hz}
'''

#### Conclusione
Questa analisi dimostra come la dimensione della finestra influisca sulla precisione dell'analisi spettrale. Finestra più grande → maggiore precisione in frequenza, ma minore risoluzione temporale.

In [ ]:
from utils.funzioni import create_sine_es
# Creiamo una sinusoide a 9093.55 Hz
sinusoide = create_sine_es(9093.55, length=1024)
fig = go.Figure()
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"))

window = 1024
spect = es.Spectrum(size = window)  # Initialize the Spectrum
sin_spec = spect(sinusoide)

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"), row=1, col=1)
fig.add_trace(go.Scatter(y=sin_spec, name = "Spettro"), row=2, col=1)
fig.show()


### Confronto tra il grafico precedente e il risultato attuale

Nel grafico attuale, possiamo osservare che la rappresentazione è meno "rumorosa" e più approssimata rispetto al grafico precedente. Questo miglioramento è dovuto alla distribuzione dei dati su un asse temporale più uniforme, che riduce la densità visiva e rende il grafico più leggibile.

#### Differenze principali:
1. **Pulizia visiva**: Il grafico attuale elimina dettagli superflui, concentrandosi sugli aspetti principali del segnale.
2. **Approssimazione**: La rappresentazione è più sintetica, ma mantiene le informazioni essenziali per l'analisi.
3. **Distribuzione temporale**: L'asse temporale è stato ottimizzato per una migliore comprensione del segnale.

Questo approccio è particolarmente utile quando si analizzano segnali complessi o di lunga durata, poiché consente di focalizzarsi sulle caratteristiche principali senza essere distratti da dettagli minori.

applicare un finestramento è facile con essentia

In [ ]:
from utils.funzioni import create_sine_es
# Creiamo una sinusoide a 9093.55 Hz
sinusoide = create_sine_es(9093.55, length=1024)
fig = go.Figure()
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"))

window = 1024
# funzioned di windowing
w = es.Windowing()

spect = es.Spectrum(size = window)  # Initialize the Spectrum
sin_spec = spect(w(sinusoide))

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=sinusoide, name="Sinusoide"), row=1, col=1)
fig.add_trace(go.Scatter(y=sin_spec, name = "Spettro"), row=2, col=1)
fig.show()


Applichiamo questa FFt al nostro audio

In [ ]:
window = 1024
sig = audio[15000:15000+window]
# funzioned di windowing
w = es.Windowing()

spect = es.Spectrum(size = window)  # Initialize the Spectrum
sin_spec = spect(w(sig))

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=audio, name="audio"), row=1, col=1)
fig.add_trace(go.Scatter(y=sin_spec, name = "Spettro"), row=2, col=1)
fig.show()


sentiamo l'audio finestrato, loopandolo con np perchè troppo breve senza

In [ ]:
IPython.display.Audio(np.tile(sig,10000), rate=sr)

Creo uno spettro

In [30]:
window = 1024
sig = audio[15000:15000+window]
# funzioned di windowing
w = es.Windowing(zeroPhase=False)
spect = es.Spectrum(size = window)  # Initialize the Spectrum
spect_arr = []


for frame in es.FrameGenerator(audio, frameSize=window, hopSize=512):
    win_spec = spect(w(frame))
    spect_arr.append(win_spec)

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=audio, name="audio"), row=1, col=1)
fig.add_trace(go.Scatter(y=spect_arr, name = "Spettro"), row=2, col=1)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

### MFCC (Mel-Frequency Cepstral Coefficients)

I Mel-Frequency Cepstral Coefficients (MFCC) sono una rappresentazione compatta e percettivamente rilevante di un segnale audio, ampiamente utilizzata nell'elaborazione del parlato e nella classificazione audio. Gli MFCC sono ottenuti applicando una serie di trasformazioni al segnale audio, che includono:

1. **Trasformata di Fourier**: Per ottenere lo spettro di frequenza del segnale.
2. **Scala Mel**: Lo spettro viene mappato su una scala Mel, che approssima la percezione umana delle frequenze.
3. **Logaritmo**: Si calcola il logaritmo dell'energia in ciascuna banda Mel.
4. **Trasformata discreta del coseno (DCT)**: Per ottenere i coefficienti finali, che rappresentano le caratteristiche principali del segnale.

#### Applicazioni
Gli MFCC sono utilizzati in numerosi campi, tra cui:
- Riconoscimento vocale
- Classificazione musicale
- Identificazione di suoni ambientali
- Analisi delle emozioni nel parlato

#### Risorse utili
- [MFCC su Wikipedia](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)
- [Documentazione ufficiale di Essentia](https://essentia.upf.edu/documentation/)
- [Introduzione agli MFCC](https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html)

### Spettro e bande Mel

Lo spettro di un segnale audio rappresenta la distribuzione delle sue frequenze. Per analizzare lo spettro in modo più simile al funzionamento del sistema uditivo umano, si utilizzano le bande Mel.

#### Bande Mel
Le bande Mel sono una rappresentazione dello spettro che approssima la percezione umana delle frequenze. Invece di considerare ogni bin dello spettro FFT, si applicano delle finestre "triangolari" che raggruppano i bin in bande di frequenza. Questo processo riduce la risoluzione spettrale, ma rende l'analisi più rilevante dal punto di vista percettivo.

#### Applicazione
Dato uno spettro con 512 bin, possiamo approssimarlo utilizzando, ad esempio, 40 bande Mel. Ogni banda rappresenta una regione di frequenze, e l'area sotto la finestra triangolare determina l'energia presente in quella banda.

In questo modo, lo spettro viene ridotto a 40 valori, ciascuno rappresentante una banda Mel, semplificando l'analisi e rendendola più affine alla percezione umana.

In [29]:
fft = spect_arr[10]
mfcc = es. MFCC(highFrequencyBound=sr//2, sampleRate=sr, numberCoefficients=14, inputSize = window//2)
# Calcoliamo i MFCC
melbands, coeff = mfcc(fft)
coeff[1:]

NameError: name 'spect_arr' is not defined

centroide spettrale

In [ ]:
cent_array=[]
cent_s_array= []
w = es.Windowing()
spect = es.Spectrum()  # Initialize the Spectrum
centroid = es.Centroid()

power = es.InstantPower()
silence = es.SilenceRate(thresholds = essentia.array([essentia.db2amp(-100)]))
silence_time = []
centroid_scaled = es.Centroid(range = 48000)
for frame in es.FrameGenerator (audio, 2048, 1024):
    p = power(frame)
    s = np.subtract(1, silence(frame))

    magnitudes = spect(w(frame))
    c = centroid(magnitudes)
    c_s = centroid_scaled(magnitudes)
    cent_array.append(c)
    cent_s_array.append(c_s)
    silence_time.append(s)

x_axis = np.linspace(0, len(audio), len(cent_array))
fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(y=audio), row=1, col=1)
fig.add_trace(go.Scatter(x= x_axis, y=np.multiply(cent_array, silence_time), name="Centroid"), row=2, col=1)
fig.add_trace(go.Scatter(x= x_axis, y=cent_s_array, name="Centroid scaled"), row=3, col=1)


# LA classe Pool permette di trattare tutti i vari array che abbiamo generato come analisi, 
# come se fosse un dizionario (cfr. struttura dati)

In [ ]:
pool = es.Pool()

In [ ]:

# Calcoliamo i MFCC
for frame in es.FrameGenerator(audio):
    mag = spect(frame)
    pool.add('magnitudes', mag)

In [ ]:
pool.remove('magnitudes') #per evitare che aggiunga e non sovrascriva i dati
for frame in es.FrameGenerator(audio):
    mag = spect(frame)
    pool.add('magnitudes', mag)


In [ ]:
pool.remove('magnitudes')

In [ ]:

pool['magnitudes']

In [ ]:
##################### ESTRATTORI ######################
# built in extractor
#per vedere solo MFCC
ext = es.Extractor(
    highLevel=False,
    midLevel=False,
    lowLevelFrameSize=1024,
    lowLevelHopSize=512,
    rhythm=False,
    relativeIoi= False
    
)
ext(audio)['lowLevel.mfcc']
es.YamlOutput(filename="ext.json", format='json')(ext(audio))


In [ ]:
pool = essentia.Pool()
# Calcoliamo i MFCC
window = 1024
w = es.Windowing(zeroPhase=False)
spect = es.Spectrum(size = window)  # Initialize the Spectrum
mfcc = es.MFCC(highFrequencyBound=sr//2, sampleRate=sr, numberCoefficients=14, inputSize = window//2)   

spect_arr = []

for frame in es.FrameGenerator(audio, frameSize=window, hopSize=512):
    win_spec = spect(w(frame))
    melbands, coeff = mfcc(win_spec)
    pool.add('mfcc', coeff[1:])
   

In [ ]:
pool['mfcc']

Aggregazione di pool

In [ ]:
aggregazione = es.PoolAggregator(defaultStats=[
    'min',
    'max',
    'mean',
    'median',
    'var',
    'stdev',
    'skew',
    'kurt',
    'dmean',
    'dvar',
    'dmean2',
    'dvar2',
    'cov', 
    'icov',
    'value'
])
es.YamlOutput(filename="mfcc.aggr.json", format='json')(aggregazione(pool))


### Formule dei descrittori statistici

Di seguito sono riportate le formule matematiche per i principali descrittori statistici utilizzati nell'analisi dei dati:

1. **Media (Mean)**  
    La media è il valore medio di un insieme di dati:
    ```math
    \text{Mean} = \frac{1}{N} \sum_{i=1}^{N} x_i
    ```

2. **Mediana (Median)**  
    La mediana è il valore centrale di un insieme di dati ordinati. Se il numero di dati è pari, è la media dei due valori centrali.

3. **Varianza (Variance)**  
    La varianza misura la dispersione dei dati rispetto alla media:
    ```math
    \text{Variance} = \frac{1}{N} \sum_{i=1}^{N} (x_i - \text{Mean})^2
    ```

4. **Deviazione standard (Standard Deviation)**  
    La deviazione standard è la radice quadrata della varianza:
    ```math
    \text{Standard Deviation} = \sqrt{\text{Variance}}
    ```

5. **Skewness (Asimmetria)**  
    Lo skewness misura l'asimmetria della distribuzione dei dati:
    ```math
    \text{Skewness} = \frac{\frac{1}{N} \sum_{i=1}^{N} (x_i - \text{Mean})^3}{\left(\sqrt{\text{Variance}}\right)^3}
    ```

6. **Curtosi (Kurtosis)**  
    La curtosi misura la "piattezza" della distribuzione dei dati:
    ```math
    \text{Kurtosis} = \frac{\frac{1}{N} \sum_{i=1}^{N} (x_i - \text{Mean})^4}{\left(\sqrt{\text{Variance}}\right)^4} - 3
    ```

7. **Media delle differenze (Delta Mean)**  
    La media delle differenze misura la variazione media tra valori consecutivi:
    ```math
    \text{Delta Mean} = \frac{1}{N-1} \sum_{i=1}^{N-1} (x_{i+1} - x_i)
    ```

8. **Varianza delle differenze (Delta Variance)**  
    La varianza delle differenze misura la dispersione delle differenze tra valori consecutivi:
    ```math
    \text{Delta Variance} = \frac{1}{N-1} \sum_{i=1}^{N-1} \left((x_{i+1} - x_i) - \text{Delta Mean}\right)^2
    ```

9. **Media delle differenze quadrate (Delta Mean Squared)**  
    La media delle differenze quadrate considera le differenze elevate al quadrato:
    ```math
    \text{Delta Mean Squared} = \frac{1}{N-1} \sum_{i=1}^{N-1} (x_{i+1} - x_i)^2
    ```

10. **Varianza delle differenze quadrate (Delta Variance Squared)**  
     La varianza delle differenze quadrate misura la dispersione delle differenze quadrate:
     ```math
     \text{Delta Variance Squared} = \frac{1}{N-1} \sum_{i=1}^{N-1} \left((x_{i+1} - x_i)^2 - \text{Delta Mean Squared}\right)^2
     ```

11. **Covarianza (Covariance)**  
     La covarianza misura la relazione lineare tra due variabili \(x\) e \(y\):
     ```math
     \text{Covariance} = \frac{1}{N} \sum_{i=1}^{N} (x_i - \text{Mean}_x)(y_i - \text{Mean}_y)
     ```

12. **Inversa della covarianza (Inverse Covariance)**  
     L'inversa della covarianza è il reciproco della covarianza, calcolata solo se la covarianza è diversa da zero:
     ```math
     \text{Inverse Covariance} = \frac{1}{\text{Covariance}}
     ```

13. **Valore (Value)**  
     Il valore rappresenta semplicemente il dato grezzo o una misura diretta senza ulteriori elaborazioni.

Queste formule sono fondamentali per descrivere e analizzare le caratteristiche statistiche di un insieme di dati.


Vatti a vedere:
1) esiste essentia nella libreria ears di MAX MSP
2) tensor flow 3.12